# Amal Sailendran, A1783111

# Global Functions

In [1]:
import numpy as np
import pandas as pd
import os
import re
import copy
from collections import Counter
# Specify git executable file for GitPython in Jupyter Notebook (In IDE, it can still work without this line.)
os.environ["GIT_PYTHON_REFRESH"] = "quiet"
from collections import Counter
import git
from git import RemoteProgress
import pprint as pprint
from git import Repo
import matplotlib.pyplot as plt
import seaborn as sns
import copy
from collections import Counter
%matplotlib inline

In [2]:
def commitsDeletionDocumented(linesModifiedDictDel,commitsDeleted):
    linesModifiedDictDel["linesDeletedAsCode"].remove(foundFlag)
    hashCode = repo.git.rev_parse(line[:line.find(" ")])
    commitsDeleted[fileName].append(hashCode)

In [3]:
def enclosingScope(line,fileComp,commitsAdded,fileName):
    indexList = int(line[line.rfind(" ", 0,line.find(")")):line.find(")")]) - 1
    individualIndex = indexList-1
    exited = False
    noSkip = True
    while individualIndex >= 0:
        lineShowData = fileComp[individualIndex][line.find(") ")+2:].strip()
        if lineShowData.endswith("}"):
            noSkip = False
        if lineShowData.endswith("{") and not noSkip:
            noSkip = True
        elif lineShowData.endswith("{") and noSkip:
            hashCode = repo.git.rev_parse(fileComp[individualIndex][:fileComp[individualIndex].find(" ")])
            commitsAdded[fileName].append(hashCode)
            exited = True
            break
        individualIndex -= 1
    if individualIndex < 0 and not exited:
        hashCode = repo.git.rev_parse(line[:line.find(" ")])
        commitsAdded[fileName].append(hashCode)

In [4]:
def printLines(commitsAddDel,commitsAddDelCount,fileName):
    for files in commitsAddDel.items():
        for commit in files:
            commitsAddDelCountFile = Counter(commit)
    commitsAddDelCount[fileName].append(commitsAddDelCountFile)
    print("File Associated: ", fileName)
    for commit, count in zip(commitsAddDelCountFile.keys(), commitsAddDelCountFile.values()):
        print("Count of Commits:", count, ", Commit:", commit)

In [5]:
def VCCCommitCount(files):
    fileNames=files[0]
    bugCommits = {}
    bugCommits[fileNames] = {}
    for file in commitsAdded[fileNames]:
        if file in commitsDeleted[fileNames] and file in commitsAdded[fileNames]:
            totalCommits = Counter(commitsDeleted[fileNames]+commitsAdded[fileNames])
        elif file in commitsDeleted[fileNames] and file not in commitsAdded[fileNames]:
            totalCommits = Counter(commitsDeleted[fileNames])
        elif file not in commitsDeleted[fileNames] and file in commitsAdded[fileNames]:
            totalCommits = Counter(commitsAdded[fileNames])
        bugCommits[fileNames][file] = {"count": 0, "commit": ""}
        for commit, count in zip(totalCommits.keys(), totalCommits.values()):
            if (count > bugCommits[fileNames][file]["count"]):
                bugCommits[fileNames][file] = {"count": count, "commit": commit}
            elif (count == bugCommits[fileNames][file]["count"]):            
                bugCommits[fileNames][file]["count"].append(count)

    print("Vulnerability Contributing Commit Count: ", bugCommits[fileNames][file]["count"], " Commit: ", bugCommits[fileNames][file]["commit"])

# 1.	CVE-ID - CVE-2012-1621
# 2.	Project Link: https://github.com/apache/ofbiz 
# 3.	Provided Fixing Commit - ea604f84e4a21fe081d66bbdab454b1e8a7d09b3 

In [6]:
repo = Repo("ofbiz/")
fixingCommit="ea604f84e4a21fe081d66bbdab454b1e8a7d09b3"
files=['applications/content/src/org/ofbiz/content/content/ContentWorker.java']
linesAdded=[]
linesDeleted=[]
buffDel=[]
buffAdd=[]
lineChanges={}
fileName=files[0]

showData = repo.git.show(fixingCommit, "--", fileName).splitlines()
for modification in showData:
    if "@@" in modification:
        line=modification
        line=line.split('@@')[1]
        line=line.replace('-','')
        line=line.replace('+','')
        line=line.replace(" ",",")
        line=line.split(",")
        buffDel=[int(line[1]),int(line[1])+int(line[2])-1]
        buffAdd=[int(line[3]),int(line[3])+int(line[4])-1]
        linesDeleted.append(buffDel)
        linesAdded.append(buffAdd)
lineChanges[files[0]] = {"Lines Added": linesAdded, "Lines Deleted": linesDeleted}

showData=repo.git.show(fixingCommit, "--", fileName).splitlines()
linesAddedAsCode=[]
linesDeletedAsCode=[]
linesModifiedDict={}
commitsDeleted = {}
commitsDeleted[fileName] = []
commitsDeletedCount = {}
commitsDeletedCount[fileName]=[]
commitsAdded = {}
commitsAdded[fileName]=[]
commitsAddedCount = {}
commitsAddedCount[fileName]=[]

for lines in showData:
    if lines.startswith('+') and not lines.startswith('+++'):
        linesAddedAsCode.append(lines)
    elif lines.startswith('-') and not lines.startswith('---'):
        linesDeletedAsCode.append(lines)
linesModifiedDict={'linesAddedAsCode':linesAddedAsCode,'linesDeletedAsCode':linesDeletedAsCode}
showBlameData=repo.git.blame(fixingCommit, "--", fileName).splitlines()
linePresent=None

print("********DELETED COMMITS********\n\n")
linesModifiedDict = copy.deepcopy(linesModifiedDict)

for linesDeleted in lineChanges[fileName]["Lines Deleted"]:
    blamedShowData = repo.git.blame(fixingCommit+"^1", "-L", str(linesDeleted[0])+","+str(linesDeleted[1]), "--", fileName).splitlines()
    for line in blamedShowData:
        blamedLineShowData = line.partition(") ")[2].strip()
        foundFlag = None
        for linesDeletedAsCode in linesModifiedDict["linesDeletedAsCode"]:
            if blamedLineShowData in linesDeletedAsCode:
                foundFlag = linesDeletedAsCode
                break
        if foundFlag != None:
            commitsDeletionDocumented(linesModifiedDict,commitsDeleted)            
printLines(commitsDeleted,commitsDeletedCount,fileName)

print("\n\n********ADDED COMMITS********\n\n")
modified_lines_copy_add = copy.deepcopy(lineChanges)
fileComp = repo.git.blame(fixingCommit, "--", fileName).splitlines()
for linesAdded in lineChanges[fileName]["Lines Added"]:
    blamedShowData = repo.git.blame(fixingCommit, "-L", str(linesAdded[0])+","+str(linesAdded[1]), "--", fileName).splitlines()
    for line in blamedShowData:
        blamedLineShowData = line.partition(") ")[2].strip()
        foundFlag = None
        for linesAddedAsCode in linesModifiedDict["linesAddedAsCode"]:
            if blamedLineShowData in linesAddedAsCode:
                foundFlag = linesAddedAsCode
                break
        if foundFlag != None:
            linesModifiedDict["linesAddedAsCode"].remove(foundFlag)
            enclosingScope(line,fileComp,commitsAdded,fileName)
printLines(commitsAdded,commitsAddedCount,fileName)
print("\n\n********VCC COMMITS********\n\n")
VCCCommitCount(files)

********DELETED COMMITS********


File Associated:  applications/content/src/org/ofbiz/content/content/ContentWorker.java
Count of Commits: 1 , Commit: 0c41fe21e77c0a5d0a1c1c0f0b47da30b7a0a47d


********ADDED COMMITS********


File Associated:  applications/content/src/org/ofbiz/content/content/ContentWorker.java
Count of Commits: 1 , Commit: 0fb6afa1b645c9d1c03e53f21436680b00e2e1af


********VCC COMMITS********


Vulnerability Contributing Commit Count:  1  Commit:  0fb6afa1b645c9d1c03e53f21436680b00e2e1af


# 1. CVE-ID - CVE-2017-15719
# 2. Project Link: https://github.com/sebfz1/wicket-jquery-ui
# 3. Provided Fixing Commit - 8aebe1e49a71f10cdd6a073fd09d0d8d82352a0

In [7]:
repo = Repo("wicket-jquery-ui/")
fixingCommit="8aebe1e49a71f10cdd6a073fd09d0d8d82352a0"
files=['wicket-jquery-ui-plugins/src/main/java/com/googlecode/wicket/jquery/ui/plugins/wysiwyg/WysiwygEditor.java']
linesAdded=[]
linesDeleted=[]
buffDel=[]
buffAdd=[]
lineChanges={}
fileName=files[0]

showData = repo.git.show(fixingCommit, "--", fileName).splitlines()
for modification in showData:
    if "@@" in modification:
        line=modification
        line=line.split('@@')[1]
        line=line.replace('-','')
        line=line.replace('+','')
        line=line.replace(" ",",")
        line=line.split(",")
        buffDel=[int(line[1]),int(line[1])+int(line[2])-1]
        buffAdd=[int(line[3]),int(line[3])+int(line[4])-1]
        linesDeleted.append(buffDel)
        linesAdded.append(buffAdd)
lineChanges[files[0]] = {"Lines Added": linesAdded, "Lines Deleted": linesDeleted}

showData=repo.git.show(fixingCommit, "--", fileName).splitlines()
linesAddedAsCode=[]
linesDeletedAsCode=[]
linesModifiedDict={}
commitsDeleted = {}
commitsDeleted[fileName] = []
commitsDeletedCount = {}
commitsDeletedCount[fileName]=[]
commitsAdded = {}
commitsAdded[fileName]=[]
commitsAddedCount = {}
commitsAddedCount[fileName]=[]

for lines in showData:
    if lines.startswith('+') and not lines.startswith('+++'):
        linesAddedAsCode.append(lines)
    elif lines.startswith('-') and not lines.startswith('---'):
        linesDeletedAsCode.append(lines)
linesModifiedDict={'linesAddedAsCode':linesAddedAsCode,'linesDeletedAsCode':linesDeletedAsCode}
showBlameData=repo.git.blame(fixingCommit, "--", fileName).splitlines()
linePresent=None

print("********DELETED COMMITS********\n\n")
linesModifiedDict = copy.deepcopy(linesModifiedDict)

for linesDeleted in lineChanges[fileName]["Lines Deleted"]:
    blamedShowData = repo.git.blame(fixingCommit+"^1", "-L", str(linesDeleted[0])+","+str(linesDeleted[1]), "--", fileName).splitlines()
    for line in blamedShowData:
        blamedLineShowData = line.partition(") ")[2].strip()
        foundFlag = None
        for linesDeletedAsCode in linesModifiedDict["linesDeletedAsCode"]:
            if blamedLineShowData in linesDeletedAsCode:
                foundFlag = linesDeletedAsCode
                break
        if foundFlag != None:
            commitsDeletionDocumented(linesModifiedDict,commitsDeleted)            
printLines(commitsDeleted,commitsDeletedCount,fileName)

print("\n\n********ADDED COMMITS********\n\n")
modified_lines_copy_add = copy.deepcopy(lineChanges)
fileComp = repo.git.blame(fixingCommit, "--", fileName).splitlines()
for linesAdded in lineChanges[fileName]["Lines Added"]:
    blamedShowData = repo.git.blame(fixingCommit, "-L", str(linesAdded[0])+","+str(linesAdded[1]), "--", fileName).splitlines()
    for line in blamedShowData:
        blamedLineShowData = line.partition(") ")[2].strip()
        foundFlag = None
        for linesAddedAsCode in linesModifiedDict["linesAddedAsCode"]:
            if blamedLineShowData in linesAddedAsCode:
                foundFlag = linesAddedAsCode
                break
        if foundFlag != None:
            linesModifiedDict["linesAddedAsCode"].remove(foundFlag)
            enclosingScope(line,fileComp,commitsAdded,fileName)
printLines(commitsAdded,commitsAddedCount,fileName)
print("\n\n********VCC COMMIT********\n\n")
VCCCommitCount(files)

********DELETED COMMITS********


File Associated:  wicket-jquery-ui-plugins/src/main/java/com/googlecode/wicket/jquery/ui/plugins/wysiwyg/WysiwygEditor.java
Count of Commits: 1 , Commit: a7b9f7961e02e2a801f5031f39e6088b49f71dea


********ADDED COMMITS********


File Associated:  wicket-jquery-ui-plugins/src/main/java/com/googlecode/wicket/jquery/ui/plugins/wysiwyg/WysiwygEditor.java
Count of Commits: 10 , Commit: 8aebe1e49a71f10cdd6a073fd09d0d8d82352a01
Count of Commits: 12 , Commit: a7b9f7961e02e2a801f5031f39e6088b49f71dea


********VCC COMMIT********


Vulnerability Contributing Commit Count:  13  Commit:  a7b9f7961e02e2a801f5031f39e6088b49f71dea


# 1.CVE-ID - CVE-2018-1337
# 2.Project Link: https://github.com/apache/directory-ldap-api
# 3.Provided Fixing Commit - 075b70a733d7af150b3d85684149ff5f029f7fd

In [8]:
repo = Repo("directory-ldap-api/")
fixingCommit="075b70a733d7af150b3d85684149ff5f029f7fd"
files=['ldap/client/api/src/main/java/org/apache/directory/ldap/client/api/LdapNetworkConnection.java']
linesAdded=[]
linesDeleted=[]
buffDel=[]
buffAdd=[]
lineChanges={}
fileName=files[0]

showData = repo.git.show(fixingCommit, "--", fileName).splitlines()
for modification in showData:
    if "@@" in modification:
        line=modification
        line=line.split('@@')[1]
        line=line.replace('-','')
        line=line.replace('+','')
        line=line.replace(" ",",")
        line=line.split(",")
        buffDel=[int(line[1]),int(line[1])+int(line[2])-1]
        buffAdd=[int(line[3]),int(line[3])+int(line[4])-1]
        linesDeleted.append(buffDel)
        linesAdded.append(buffAdd)
lineChanges[files[0]] = {"Lines Added": linesAdded, "Lines Deleted": linesDeleted}

showData=repo.git.show(fixingCommit, "--", fileName).splitlines()
linesAddedAsCode=[]
linesDeletedAsCode=[]
linesModifiedDict={}
commitsDeleted = {}
commitsDeleted[fileName] = []
commitsDeletedCount = {}
commitsDeletedCount[fileName]=[]
commitsAdded = {}
commitsAdded[fileName]=[]
commitsAddedCount = {}
commitsAddedCount[fileName]=[]

for lines in showData:
    if lines.startswith('+') and not lines.startswith('+++'):
        linesAddedAsCode.append(lines)
    elif lines.startswith('-') and not lines.startswith('---'):
        linesDeletedAsCode.append(lines)
linesModifiedDict={'linesAddedAsCode':linesAddedAsCode,'linesDeletedAsCode':linesDeletedAsCode}
showBlameData=repo.git.blame(fixingCommit, "--", fileName).splitlines()
linePresent=None

print("********DELETED COMMITS********\n\n")
linesModifiedDict = copy.deepcopy(linesModifiedDict)

for linesDeleted in lineChanges[fileName]["Lines Deleted"]:
    blamedShowData = repo.git.blame(fixingCommit+"^1", "-L", str(linesDeleted[0])+","+str(linesDeleted[1]), "--", fileName).splitlines()
    for line in blamedShowData:
        blamedLineShowData = line.partition(") ")[2].strip()
        foundFlag = None
        for linesDeletedAsCode in linesModifiedDict["linesDeletedAsCode"]:
            if blamedLineShowData in linesDeletedAsCode:
                foundFlag = linesDeletedAsCode
                break
        if foundFlag != None:
            commitsDeletionDocumented(linesModifiedDict,commitsDeleted)            
printLines(commitsDeleted,commitsDeletedCount,fileName)

print("\n\n********ADDED COMMITS********\n\n")
modified_lines_copy_add = copy.deepcopy(lineChanges)
fileComp = repo.git.blame(fixingCommit, "--", fileName).splitlines()
for linesAdded in lineChanges[fileName]["Lines Added"]:
    blamedShowData = repo.git.blame(fixingCommit, "-L", str(linesAdded[0])+","+str(linesAdded[1]), "--", fileName).splitlines()
    for line in blamedShowData:
        blamedLineShowData = line.partition(") ")[2].strip()
        foundFlag = None
        for linesAddedAsCode in linesModifiedDict["linesAddedAsCode"]:
            if blamedLineShowData in linesAddedAsCode:
                foundFlag = linesAddedAsCode
                break
        if foundFlag != None:
            linesModifiedDict["linesAddedAsCode"].remove(foundFlag)
            enclosingScope(line,fileComp,commitsAdded,fileName)
printLines(commitsAdded,commitsAddedCount,fileName)
print("\n\n********VCC COMMIT********\n\n")
VCCCommitCount(files)

********DELETED COMMITS********


File Associated:  ldap/client/api/src/main/java/org/apache/directory/ldap/client/api/LdapNetworkConnection.java
Count of Commits: 2 , Commit: fc948ea0675705d7efaf1014551e278b448c6e2e
Count of Commits: 3 , Commit: d127ce2688a77800d1a941c4aafcfa1bf23be279
Count of Commits: 1 , Commit: 634bc969bdb0d796cc6ab52b154a1d3ff84f4779
Count of Commits: 2 , Commit: 826a0a51fb44e8209e40fdf9584d9dccc6e28021
Count of Commits: 1 , Commit: 728acfcdf0dcd879cc1a0370a28d014133968103
Count of Commits: 15 , Commit: 61ef7c1f2ff6e60cdf1e999cc6c72fe55e72de5a
Count of Commits: 2 , Commit: 4fcb60ba96fe15ee4d1ad9edf7fca640258bf33d
Count of Commits: 4 , Commit: 1344866d49ed71c5901e0af1eb266cd01032c680
Count of Commits: 1 , Commit: 09f5b1d8784662c642afaaed11b682299b60caeb
Count of Commits: 1 , Commit: 85d503945cc27008ea6d074e8096403a5bd97c0c
Count of Commits: 4 , Commit: c2cebdaa13dbe41b5fea393fc588cb1807ca1baa


********ADDED COMMITS********


File Associated:  ldap/client/api/src/